<a href="https://colab.research.google.com/github/fbeilstein/presentations/blob/master/presentation_for_June_27_2023_lecture_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##Back propagation (how to get gradient)
%%html

<style>
  .MathJax {
font-size: 0.6em;
}
</style>
<script src='https://www.gstatic.com/external_hosted/mathjax/latest/MathJax.js?config=default'>
</script>

<div id="slide_1" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_1.jpg" width="750px">
</div>

<div id="slide_2" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_2.jpg" width="750px">
</div>

<div id="slide_3" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_3.jpg" width="750px">
</div>

<div id="slide_4" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_4.jpg" width="750px">
</div>

<div id="slide_5" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_5.jpg" width="750px">
</div>

<div id="slide_6" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_6.jpg" width="750px">
</div>

<div id="slide_7" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_7.jpg" width="750px">
</div>

<div id="slide_8" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_8.jpg" width="750px">
</div>

<div id="slide_9" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_9.jpg" width="750px">
</div>

<div id="slide_10" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_10.jpg" width="750px">
</div>

<div id="slide_11" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_11.jpg" width="750px">
</div>

<div id="slide_12" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_12.jpg" width="750px">
</div>

<div id="slide_13" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_13.jpg" width="750px">
</div>

<div id="slide_14" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_14.jpg" width="750px">
</div>

<div id="slide_15" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_15.jpg" width="750px">
</div>

<div id="slide_16" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_16.jpg" width="750px">
</div>

<div id="slide_17" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_17.jpg" width="750px">
</div>

<div id="slide_18" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_18.jpg" width="750px">
</div>

<div id="slide_19" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_19.jpg" width="750px">
</div>

<div id="slide_20" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_20.jpg" width="750px">
</div>


<div style="position:absolute; top:100px; left:800px;">

<div id="arrow_left" style="border-width: 1px; border-style:solid; float:left; height:32px;">
<svg width="8.7464mm" height="8.7464mm" version="1.1" viewBox="0 0 8.7464 8.7464" xmlns="http://www.w3.org/2000/svg" xmlns:cc="http://creativecommons.org/ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
<metadata>
<rdf:RDF>
<cc:Work rdf:about="">
<dc:format>image/svg+xml</dc:format>
<dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
<dc:title/>
</cc:Work>
</rdf:RDF>
</metadata>
<g transform="translate(-38.564 -29.908)">
<rect x="38.564" y="29.908" width="8.7464" height="8.7464" fill-opacity=".16425" opacity=".97" stroke-miterlimit="10.433" stroke-width="0"/>
<path d="m45.796 34.331h-5.8931" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m40.068 34.267 2.7141-2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m40.069 34.396 2.7141 2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
</g>
</svg>
</div>

<div id="arrow_right" style="border-width: 1px; border-style:solid; float:right; height:32px;">
<svg width="8.7464mm" height="8.7464mm" version="1.1" viewBox="0 0 8.7464 8.7464" xmlns="http://www.w3.org/2000/svg" xmlns:cc="http://creativecommons.org/ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
<metadata>
<rdf:RDF>
<cc:Work rdf:about="">
<dc:format>image/svg+xml</dc:format>
<dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
<dc:title/>
</cc:Work>
</rdf:RDF>
</metadata>
<g transform="translate(-38.564 -29.908)">
<rect transform="scale(-1,1)" x="-47.31" y="29.908" width="8.7464" height="8.7464" fill-opacity=".16425" opacity=".97" stroke-miterlimit="10.433" stroke-width="0"/>
<path d="m40.078 34.331h5.8931" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m45.806 34.267-2.7141-2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m45.805 34.396-2.7141 2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
</g>
</svg>
</div>

</div>

<style>
#arrow_left:hover{color:blue;}
#arrow_right:hover{color:blue;}
</style>

<script  type="text/javascript">
var slides = ["slide_1", "slide_2", "slide_3", "slide_4",
               "slide_5", "slide_6",
              "slide_7", "slide_8", "slide_9",
              "slide_10", "slide_11", "slide_12",
              "slide_13", "slide_14", "slide_15",
              "slide_16", "slide_17", "slide_18",
              "slide_19", "slide_20"];
var left_btn_id = "arrow_left";
var right_btn_id = "arrow_right";
var current_slide = 0;
var merging = false;

function slide_left()
{
  var old_slide = document.getElementById(slides[current_slide]);
  current_slide -= 1;
  if (current_slide < 0) {current_slide = 0;}
  var new_slide = document.getElementById(slides[current_slide]);
  old_slide.style.display='none';
  new_slide.style.display='block';
  if (current_slide != 5)
      merging = false;
}

function slide_right()
{
  var old_slide = document.getElementById(slides[current_slide]);
  current_slide += 1;
  if (current_slide >= slides.length) {current_slide = slides.length - 1;}
  var new_slide = document.getElementById(slides[current_slide]);
  old_slide.style.display='none';
  new_slide.style.display='block';
  if (current_slide == 5)
  {
    if (!merging)
      merge_images(200);
    merging = true;
  } else {
    merging = false;
  }
}

function image_opacity(image, opacity)
{
    var s = image.style;
    s.opacity = (opacity / 100);
    s.MozOpacity = (opacity / 100);
    s.KhtmlOpacity = (opacity / 100);
    s.filter = 'alpha(opacity=' + opacity + ')';
}

function merge_images(cycles_left)
{
  document.getElementById("cat").style.left = (200 - cycles_left) + "px";
  document.getElementById("pic").style.left = (400 - (200 - cycles_left)) + "px";
  image_opacity(document.getElementById("cat"), cycles_left / 2)
  image_opacity(document.getElementById("pic"), cycles_left / 2)
  image_opacity(document.getElementById("merge"), 100 - cycles_left / 2)
  if (cycles_left > 0)
    setTimeout(function(){merge_images(cycles_left - 1);}, 3);
}

document.getElementById(left_btn_id).addEventListener("click", slide_left);
document.getElementById(right_btn_id).addEventListener("click", slide_right);
document.getElementById(slides[current_slide]).style.display='block';
</script>

In [3]:
#@title ##NN example
%%html

<style>
  .MathJax {
font-size: 0.6em;
}
</style>
<script src='https://www.gstatic.com/external_hosted/mathjax/latest/MathJax.js?config=default'>
</script>

<div id="slide_1" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_1.jpg" width="750px">
</div>

<div id="slide_2" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_2.jpg" width="750px">
</div>

<div id="slide_3" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_3.jpg" width="750px">
</div>

<div id="slide_4" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_4.jpg" width="750px">
</div>

<div id="slide_5" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_5.jpg" width="750px">
</div>

<div id="slide_6" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_6.jpg" width="750px">
</div>

<div id="slide_7" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_7.jpg" width="750px">
</div>

<div id="slide_8" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_8.jpg" width="750px">
</div>

<div id="slide_9" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_9.jpg" width="750px">
</div>

<div id="slide_10" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_10.jpg" width="750px">
</div>

<div id="slide_11" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_11.jpg" width="750px">
</div>

<div id="slide_12" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_12.jpg" width="750px">
</div>

<div id="slide_13" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_13.jpg" width="750px">
</div>

<div id="slide_14" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_14.jpg" width="750px">
</div>

<div id="slide_15" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_15.jpg" width="750px">
</div>

<div id="slide_16" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_16.jpg" width="750px">
</div>

<div id="slide_17" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_17.jpg" width="750px">
</div>

<div id="slide_18" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_18.jpg" width="750px">
</div>

<div id="slide_19" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_19.jpg" width="750px">
</div>

<div id="slide_20" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_20.jpg" width="750px">
</div>


<div style="position:absolute; top:100px; left:800px;">

<div id="arrow_left" style="border-width: 1px; border-style:solid; float:left; height:32px;">
<svg width="8.7464mm" height="8.7464mm" version="1.1" viewBox="0 0 8.7464 8.7464" xmlns="http://www.w3.org/2000/svg" xmlns:cc="http://creativecommons.org/ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
<metadata>
<rdf:RDF>
<cc:Work rdf:about="">
<dc:format>image/svg+xml</dc:format>
<dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
<dc:title/>
</cc:Work>
</rdf:RDF>
</metadata>
<g transform="translate(-38.564 -29.908)">
<rect x="38.564" y="29.908" width="8.7464" height="8.7464" fill-opacity=".16425" opacity=".97" stroke-miterlimit="10.433" stroke-width="0"/>
<path d="m45.796 34.331h-5.8931" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m40.068 34.267 2.7141-2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m40.069 34.396 2.7141 2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
</g>
</svg>
</div>

<div id="arrow_right" style="border-width: 1px; border-style:solid; float:right; height:32px;">
<svg width="8.7464mm" height="8.7464mm" version="1.1" viewBox="0 0 8.7464 8.7464" xmlns="http://www.w3.org/2000/svg" xmlns:cc="http://creativecommons.org/ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
<metadata>
<rdf:RDF>
<cc:Work rdf:about="">
<dc:format>image/svg+xml</dc:format>
<dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
<dc:title/>
</cc:Work>
</rdf:RDF>
</metadata>
<g transform="translate(-38.564 -29.908)">
<rect transform="scale(-1,1)" x="-47.31" y="29.908" width="8.7464" height="8.7464" fill-opacity=".16425" opacity=".97" stroke-miterlimit="10.433" stroke-width="0"/>
<path d="m40.078 34.331h5.8931" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m45.806 34.267-2.7141-2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m45.805 34.396-2.7141 2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
</g>
</svg>
</div>

</div>

<style>
#arrow_left:hover{color:blue;}
#arrow_right:hover{color:blue;}
</style>

<script  type="text/javascript">
var slides = ["slide_1", "slide_2", "slide_3", "slide_4",
               "slide_5", "slide_6",
              "slide_7", "slide_8", "slide_9",
              "slide_10", "slide_11", "slide_12",
              "slide_13", "slide_14", "slide_15",
              "slide_16", "slide_17", "slide_18",
              "slide_19", "slide_20"];
var left_btn_id = "arrow_left";
var right_btn_id = "arrow_right";
var current_slide = 0;
var merging = false;

function slide_left()
{
  var old_slide = document.getElementById(slides[current_slide]);
  current_slide -= 1;
  if (current_slide < 0) {current_slide = 0;}
  var new_slide = document.getElementById(slides[current_slide]);
  old_slide.style.display='none';
  new_slide.style.display='block';
  if (current_slide != 5)
      merging = false;
}

function slide_right()
{
  var old_slide = document.getElementById(slides[current_slide]);
  current_slide += 1;
  if (current_slide >= slides.length) {current_slide = slides.length - 1;}
  var new_slide = document.getElementById(slides[current_slide]);
  old_slide.style.display='none';
  new_slide.style.display='block';
  if (current_slide == 5)
  {
    if (!merging)
      merge_images(200);
    merging = true;
  } else {
    merging = false;
  }
}

function image_opacity(image, opacity)
{
    var s = image.style;
    s.opacity = (opacity / 100);
    s.MozOpacity = (opacity / 100);
    s.KhtmlOpacity = (opacity / 100);
    s.filter = 'alpha(opacity=' + opacity + ')';
}

function merge_images(cycles_left)
{
  document.getElementById("cat").style.left = (200 - cycles_left) + "px";
  document.getElementById("pic").style.left = (400 - (200 - cycles_left)) + "px";
  image_opacity(document.getElementById("cat"), cycles_left / 2)
  image_opacity(document.getElementById("pic"), cycles_left / 2)
  image_opacity(document.getElementById("merge"), 100 - cycles_left / 2)
  if (cycles_left > 0)
    setTimeout(function(){merge_images(cycles_left - 1);}, 3);
}

document.getElementById(left_btn_id).addEventListener("click", slide_left);
document.getElementById(right_btn_id).addEventListener("click", slide_right);
document.getElementById(slides[current_slide]).style.display='block';
</script>

In [2]:
#@title ##GAN style transfer
%%html


<div id="slide_6" style="display:none; max-width:750px; height:200px;">
  <img src="https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_01_introduction/Cat.jpg"
  style="position:absolute; left:0px;  top:0px;">
  <img src="https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_01_introduction/Candinsky.jpg"
  style="position:absolute; left:400px; top:0px;">
</div>

<div id="slide_7" style="display:none; max-width:750px; height:200px;">
  <img src="https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_01_introduction/Cat.jpg"
  style="position:absolute; left:0px;  top:0px;" id="cat">
  <img src="https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_01_introduction/CandinskyCat.jpg"
  style="position:absolute; left:200px; top:0px;" id="merge">
  <img src="https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_01_introduction/Candinsky.jpg"
  style="position:absolute; left:400px; top:0px;" id="pic">
</div>



<div style="position:absolute; top:100px; left:800px;">

<div id="arrow_left" style="border-width: 1px; border-style:solid; float:left; height:32px;">
<svg width="8.7464mm" height="8.7464mm" version="1.1" viewBox="0 0 8.7464 8.7464" xmlns="http://www.w3.org/2000/svg" xmlns:cc="http://creativecommons.org/ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
<metadata>
<rdf:RDF>
<cc:Work rdf:about="">
<dc:format>image/svg+xml</dc:format>
<dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
<dc:title/>
</cc:Work>
</rdf:RDF>
</metadata>
<g transform="translate(-38.564 -29.908)">
<rect x="38.564" y="29.908" width="8.7464" height="8.7464" fill-opacity=".16425" opacity=".97" stroke-miterlimit="10.433" stroke-width="0"/>
<path d="m45.796 34.331h-5.8931" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m40.068 34.267 2.7141-2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m40.069 34.396 2.7141 2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
</g>
</svg>
</div>

<div id="arrow_right" style="border-width: 1px; border-style:solid; float:right; height:32px;">
<svg width="8.7464mm" height="8.7464mm" version="1.1" viewBox="0 0 8.7464 8.7464" xmlns="http://www.w3.org/2000/svg" xmlns:cc="http://creativecommons.org/ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
<metadata>
<rdf:RDF>
<cc:Work rdf:about="">
<dc:format>image/svg+xml</dc:format>
<dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
<dc:title/>
</cc:Work>
</rdf:RDF>
</metadata>
<g transform="translate(-38.564 -29.908)">
<rect transform="scale(-1,1)" x="-47.31" y="29.908" width="8.7464" height="8.7464" fill-opacity=".16425" opacity=".97" stroke-miterlimit="10.433" stroke-width="0"/>
<path d="m40.078 34.331h5.8931" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m45.806 34.267-2.7141-2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m45.805 34.396-2.7141 2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
</g>
</svg>
</div>

</div>

<style>
#arrow_left:hover{color:blue;}
#arrow_right:hover{color:blue;}
</style>

<script  type="text/javascript">
var slides = ["slide_6", "slide_7"];
var left_btn_id = "arrow_left";
var right_btn_id = "arrow_right";
var current_slide = 0;
var merging = false;

function slide_left()
{
  var old_slide = document.getElementById(slides[current_slide]);
  current_slide -= 1;
  if (current_slide < 0) {current_slide = 0;}
  var new_slide = document.getElementById(slides[current_slide]);
  old_slide.style.display='none';
  new_slide.style.display='block';
  if (current_slide != 5)
      merging = false;
}

function slide_right()
{
  var old_slide = document.getElementById(slides[current_slide]);
  current_slide += 1;
  if (current_slide >= slides.length) {current_slide = slides.length - 1;}
  var new_slide = document.getElementById(slides[current_slide]);
  old_slide.style.display='none';
  new_slide.style.display='block';
  if (current_slide == 1)
  {
    if (!merging)
      merge_images(200);
    merging = true;
  } else {
    merging = false;
  }
}

function image_opacity(image, opacity)
{
    var s = image.style;
    s.opacity = (opacity / 100);
    s.MozOpacity = (opacity / 100);
    s.KhtmlOpacity = (opacity / 100);
    s.filter = 'alpha(opacity=' + opacity + ')';
}

function merge_images(cycles_left)
{
  document.getElementById("cat").style.left = (200 - cycles_left) + "px";
  document.getElementById("pic").style.left = (400 - (200 - cycles_left)) + "px";
  image_opacity(document.getElementById("cat"), cycles_left / 2)
  image_opacity(document.getElementById("pic"), cycles_left / 2)
  image_opacity(document.getElementById("merge"), 100 - cycles_left / 2)
  if (cycles_left > 0)
    setTimeout(function(){merge_images(cycles_left - 1);}, 3);
}

document.getElementById(left_btn_id).addEventListener("click", slide_left);
document.getElementById(right_btn_id).addEventListener("click", slide_right);
document.getElementById(slides[current_slide]).style.display='block';
</script>